In [1]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt


def get_dataset_from_testcase(run):
    return run[0:-2]

def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

runs = []
dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/join_all'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/string_keys'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'dataset_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [2]:
def get_index_type(index):
    if "btree" in index:
        return "BTREE"
    if "pgm" in index:
        return "PGM"
    return "NA"

def get_index_variant(index):
    if index == "sort_join" or index == "hash_join":
        return "NA"
    pos = index.find("_")
    return index[pos+1:-1]

df["threads"] = df["spec.num_threads"]
df["epsilon"] = df["spec.index.epsilon"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["thput"] = (df["result.num_output_keys"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["index_type"] = df["spec.algo_name"].map(lambda x: get_index_type(x))
df["index_variant"] = df["spec.algo_name"].map(lambda x: get_index_variant(x))
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
display(df["algo"].unique())
display(df["dataset"].unique())
display(df["index_type"].unique())


array(['lsj_sampledflatpgm256', 'inlj_flatpgm4096', 'lsj_btree4096',
       'inlj_sampledflatpgm4096', 'inlj_flatpgm1024', 'hash_join',
       'inlj_pgm4096', 'sort_join', 'inlj_pgm1024', 'inlj_btree1024',
       'lsj_sampledflatpgm1024', 'inlj_btree256',
       'inlj_sampledflatpgm1024', 'lsj_pgm256', 'lsj_btree1024',
       'lsj_btree256', 'inlj_flatpgm256', 'lsj_flatpgm1024',
       'inlj_sampledflatpgm256', 'lsj_sampledflatpgm4096', 'inlj_pgm256',
       'lsj_pgm4096', 'lsj_pgm1024', 'lsj_flatpgm4096', 'lsj_flatpgm256',
       'inlj_btree4096'], dtype=object)

array(['fb', 'books', 'osm', 'uniform_dense', 'lognormal',
       'uniform_sparse', 'normal', 'wiki', 'string_keys'], dtype=object)

array(['PGM', 'BTREE', 'NA'], dtype=object)

In [6]:
# SingleThread, HJ vs SJ vs INLJ(BTree256) vs INLJ(PGM256) vs LS(BTree256) vs LS(PGM256)
def plot_dataset_join_duration(dataset):
    rows = duckdb.sql(
        "SELECT ratio, threads, algo, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   (algo='hash_join' OR algo='sort_join' OR algo='inlj_btree256' OR algo='inlj_pgm256' OR "
        "   algo='lsj_btree256' OR algo='lsj_flatpgm256')" 
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, threads, algo"
    ).df()
    return(rows.pivot(index='ratio', values='d', columns=['algo']))

real_datasets = ['fb', 'wiki', 'osm', 'books']
synth_datasets = ['uniform_dense', 'uniform_sparse', 'normal', 'lognormal', 'string_keys']

for dataset in real_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

'fb'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,92.803386,53.067202,39.427941,47.774045,25.976550,15.907270
10,49.446850,6.178218,4.912952,6.495105,4.075511,4.825520
100,37.227181,1.664028,1.084884,1.634213,1.199437,3.364797
1000,14.757411,0.259721,0.352090,0.315716,0.285341,3.176952


'wiki'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,37.334655,23.540610,15.039791,20.846256,11.690900,6.969193
10,22.695847,5.059676,3.505500,4.941911,3.507018,2.795611
100,16.403791,0.875864,1.013105,0.882262,0.615757,1.422261
1000,6.730158,0.222480,0.223985,0.222210,0.216568,1.245280


'osm'

...

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,94.056723,54.062095,29.698941,47.962588,26.738348,16.558630
10,47.265245,6.727578,3.945524,5.999719,3.541300,5.026393
100,37.498558,1.192461,0.938083,1.193659,1.072242,2.894747
1000,14.713410,0.243950,0.312992,0.305125,0.305724,2.695198


'lognormal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,91.594071,54.251634,33.305641,48.218088,24.873390,17.354602
10,50.659646,6.138937,4.297023,6.553460,4.182600,4.595661
100,39.186771,1.189923,1.006116,1.191112,1.075444,2.879593
1000,15.736142,0.239541,0.258540,0.303619,0.306435,2.703660


'string_keys'

algo,hash_join,inlj_btree256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,
1,37.564923,52.573234,52.511865,11.771871,5.942693
10,37.489340,9.813894,9.920805,2.606203,2.032171
100,23.681283,1.318502,1.312316,0.505042,1.126635
1000,9.822884,0.236980,0.235829,0.164468,1.005547


In [7]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_inlj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'inlj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

'fb'

algo,inlj_flatpgm4096,inlj_sampledflatpgm4096,inlj_btree1024,inlj_btree4096,inlj_btree256,inlj_pgm256,inlj_flatpgm1024,inlj_flatpgm256,inlj_pgm1024,inlj_sampledflatpgm256,inlj_sampledflatpgm1024,inlj_pgm4096
epsilon,4096.0,4096.0,1024.0,4096.0,256.0,256.0,1024.0,256.0,1024.0,256.0,1024.0,4096.0
ratio,,,,,,,,,,,,
1,36.860210,37.738168,54.454780,53.186117,53.067202,39.427941,36.694180,35.735968,38.800283,39.077775,39.386363,42.717257
10,5.570609,5.405943,6.049074,5.819146,6.178218,4.912952,5.253616,4.782187,5.297466,5.984620,5.623032,5.850861
100,1.833962,1.536952,1.130245,1.093045,1.664028,1.084884,1.593616,1.418238,1.199611,1.619190,1.396686,1.413768
1000,0.775226,0.645554,0.398953,0.510789,0.259721,0.352090,0.485742,0.346613,0.491207,0.377439,0.427322,0.787943


'wiki'

algo,inlj_sampledflatpgm4096,inlj_btree1024,inlj_sampledflatpgm1024,inlj_btree4096,inlj_flatpgm4096,inlj_pgm256,inlj_btree256,inlj_sampledflatpgm256,inlj_pgm1024,inlj_pgm4096,inlj_flatpgm256,inlj_flatpgm1024
epsilon,4096.0,1024.0,1024.0,4096.0,4096.0,256.0,256.0,256.0,1024.0,4096.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,17.307899,22.996426,16.447396,21.834449,15.443946,15.039791,23.540610,15.234307,15.998415,16.979243,13.750013,14.622424
10,4.318298,5.454602,4.215259,4.802448,3.783417,3.505500,5.059676,4.056870,3.834758,4.059837,3.788520,3.527932
100,1.062199,0.822400,1.003368,0.808017,0.932598,1.013105,0.875864,0.931938,0.840494,0.964979,0.718000,0.812739
1000,0.424065,0.276095,0.316107,0.292301,0.400498,0.223985,0.222480,0.306665,0.293112,0.404468,0.222146,0.288934


'osm'

...

algo,inlj_flatpgm1024,inlj_btree1024,inlj_btree256,inlj_flatpgm256,inlj_btree4096,inlj_pgm4096,inlj_flatpgm4096,inlj_sampledflatpgm1024,inlj_sampledflatpgm4096,inlj_pgm1024,inlj_pgm256,inlj_sampledflatpgm256
epsilon,1024.0,1024.0,256.0,256.0,4096.0,4096.0,4096.0,1024.0,4096.0,1024.0,256.0,256.0
ratio,,,,,,,,,,,,
1,29.423931,56.033252,54.062095,27.696246,53.901132,35.679257,31.833445,34.092265,35.685901,32.682569,29.698941,30.905788
10,4.082723,6.414867,6.727578,3.720996,6.607661,5.019567,5.038547,5.635330,6.696616,4.791668,3.945524,5.585452
100,1.510040,1.462578,1.192461,0.913872,1.394548,1.924672,1.433930,1.748598,2.257273,1.569407,0.938083,1.640410
1000,0.422447,0.424038,0.243950,0.306580,0.538436,0.732859,0.734382,0.382800,0.582767,0.360370,0.312992,0.278286


'lognormal'

algo,inlj_pgm1024,inlj_btree256,inlj_btree1024,inlj_flatpgm256,inlj_sampledflatpgm256,inlj_pgm4096,inlj_pgm256,inlj_sampledflatpgm4096,inlj_btree4096,inlj_sampledflatpgm1024,inlj_flatpgm4096,inlj_flatpgm1024
epsilon,1024.0,256.0,1024.0,256.0,256.0,4096.0,256.0,4096.0,4096.0,1024.0,4096.0,1024.0
ratio,,,,,,,,,,,,
1,32.894151,54.251634,55.068840,27.484243,31.509494,37.297779,33.305641,36.831732,52.642450,34.070140,32.922510,29.172672
10,4.741701,6.138937,6.585124,3.737744,5.766262,6.134477,4.297023,7.009174,5.840396,6.068160,5.242064,4.229369
100,1.235123,1.189923,1.449490,0.945974,1.258313,2.442943,1.006116,2.254006,1.015438,1.535502,1.744232,1.184007
1000,0.373348,0.239541,0.427705,0.310651,0.339300,0.837495,0.258540,0.843014,0.438325,0.400146,0.622090,0.405674


'string_keys'

algo,inlj_btree1024,inlj_btree4096,inlj_flatpgm4096,inlj_flatpgm1024,inlj_btree256,inlj_flatpgm256
epsilon,1024.0,4096.0,4096.0,1024.0,256.0,256.0
ratio,,,,,,
1,41.705262,38.333900,15.275076,14.301378,52.573234,13.383711
10,7.804917,7.191015,3.333016,3.106199,9.813894,2.736875
100,1.051437,0.974296,0.767426,0.689211,1.318502,0.599349
1000,0.295717,0.284786,0.351275,0.241957,0.236980,0.169305


In [8]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_lsj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'lsj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

'fb'

algo,lsj_btree1024,lsj_pgm4096,lsj_pgm1024,lsj_sampledflatpgm4096,lsj_flatpgm1024,lsj_sampledflatpgm256,lsj_flatpgm4096,lsj_flatpgm256,lsj_btree4096,lsj_btree256,lsj_sampledflatpgm1024,lsj_pgm256
epsilon,1024.0,4096.0,1024.0,4096.0,1024.0,256.0,4096.0,256.0,4096.0,256.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,49.462819,36.605862,31.165740,31.556395,28.265467,27.388705,30.195188,25.976550,48.104614,47.774045,29.109658,32.369930
10,6.081613,5.273251,4.436041,4.614040,4.108070,3.827594,4.621865,4.075511,6.269327,6.495105,3.999292,4.987520
100,1.097516,1.284478,0.928397,0.882036,1.137769,0.911438,0.892550,1.199437,1.510024,1.634213,0.857476,0.960022
1000,0.400823,0.572332,0.406372,0.572286,0.460544,0.309895,0.468081,0.285341,0.618737,0.315716,0.401034,0.356856


'wiki'

algo,lsj_pgm256,lsj_sampledflatpgm256,lsj_pgm1024,lsj_flatpgm4096,lsj_flatpgm256,lsj_sampledflatpgm4096,lsj_sampledflatpgm1024,lsj_btree4096,lsj_pgm4096,lsj_btree256,lsj_btree1024,lsj_flatpgm1024
epsilon,256.0,256.0,1024.0,4096.0,256.0,4096.0,1024.0,4096.0,4096.0,256.0,1024.0,1024.0
ratio,,,,,,,,,,,,
1,13.718574,12.475902,13.968240,13.896093,11.690900,14.032697,13.001416,19.928272,15.630617,20.846256,20.547359,12.601081
10,3.420263,3.153455,3.549318,3.437761,3.507018,3.278784,3.232015,4.678248,3.804960,4.941911,4.782398,3.158138
100,0.658492,0.634182,0.897737,0.640505,0.615757,0.649068,0.625782,0.806055,0.688598,0.882262,0.805663,0.808706
1000,0.224960,0.241131,0.258776,0.270140,0.216568,0.273742,0.258993,0.353141,0.275745,0.222210,0.275110,0.258626


'osm'

...

algo,lsj_btree4096,lsj_sampledflatpgm1024,lsj_btree256,lsj_flatpgm1024,lsj_pgm256,lsj_pgm1024,lsj_flatpgm4096,lsj_sampledflatpgm4096,lsj_btree1024,lsj_sampledflatpgm256,lsj_flatpgm256,lsj_pgm4096
epsilon,4096.0,1024.0,256.0,1024.0,256.0,1024.0,4096.0,4096.0,1024.0,256.0,256.0,4096.0
ratio,,,,,,,,,,,,
1,49.517564,29.161457,47.962588,28.126468,28.804077,30.955543,30.564132,31.174071,49.879021,27.793841,26.738348,33.169340
10,6.012182,3.981949,5.999719,4.123303,4.087392,4.095853,4.554804,4.211037,6.585157,4.347632,3.541300,4.567865
100,1.409810,1.078725,1.193659,0.789183,0.860061,0.843884,1.091609,1.118884,1.047576,0.860416,1.072242,0.856568
1000,0.447122,0.366350,0.305125,0.414492,0.314354,0.414432,0.414343,0.492297,0.438340,0.332130,0.305724,0.506079


'lognormal'

algo,lsj_btree4096,lsj_btree1024,lsj_flatpgm4096,lsj_sampledflatpgm1024,lsj_flatpgm256,lsj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm4096,lsj_pgm4096,lsj_pgm1024,lsj_pgm256,lsj_flatpgm1024
epsilon,4096.0,1024.0,4096.0,1024.0,256.0,256.0,256.0,4096.0,4096.0,1024.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,48.454052,49.287141,29.567297,29.083425,24.873390,27.162494,48.218088,30.776425,33.292755,31.035748,31.149820,27.160560
10,5.886518,5.887685,4.608930,4.267458,4.182600,4.385090,6.553460,4.145133,4.779192,4.256476,4.208748,4.426708
100,1.011300,1.025885,1.079935,0.812445,1.075444,0.870240,1.191112,1.098953,0.837151,0.829662,0.909516,0.802930
1000,0.529026,0.340028,0.404458,0.352382,0.306435,0.265743,0.303619,0.501061,0.415363,0.434322,0.256857,0.419654


'string_keys'

algo,lsj_btree4096,lsj_btree1024,lsj_flatpgm1024,lsj_flatpgm4096,lsj_btree256,lsj_flatpgm256
epsilon,4096.0,1024.0,1024.0,4096.0,256.0,256.0
ratio,,,,,,
1,37.402234,40.859599,12.444793,14.217743,52.511865,11.771871
10,7.170656,7.707481,2.657849,2.946982,9.920805,2.606203
100,0.980559,1.053253,0.509415,0.521806,1.312316,0.505042
1000,0.277917,0.284380,0.212594,0.227511,0.235829,0.164468
